<a href="https://colab.research.google.com/github/dolmani38/drift/blob/main/Concept%20drift%EC%97%90%20%EC%9D%98%ED%95%9C%20%EB%AA%A8%EB%8D%B8%EC%9D%98%20%EC%84%B1%EB%8A%A5%20%EC%B6%94%EC%A0%95%20%EB%B0%A9%EB%B2%95_0601.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

*   기계학습은 학습에 의해 모델을 구축한 후, Test dataset을 사용하여 N-fold class-validation으로 precision, recall, f1 및 MAPE, RMSE 등 성능을 측정한다.
*   통상, 측정된 성능값은 해당 모델을 대변하는 성능 지표가 된다.
*   이렇게 구축된 모델은 live data를 입력해서 실 활용하게 되는데, 이때 초기에 측정한 성능을 지속하여 확보하고 있는지 실측하여 알 수 없다.
*   Time-series data에 의한 Regression의 경우, 예측 기간의 도래 이후, 실측 Y값을 알 수 있으나 역시 업무에 따라 그 시간 간격이 크게 발생 할 수 있으며, classification의 경우, 예측 class가 정확한지 실측하기 위해서는 별도의 labeling 작업을 해야 한다.
*   즉, 한번 구축한 모델은 활용 단계에서 성능을 실측하는 것이 매우 어려우며, 비즈니스상에서 IT운영팀에서 이를 관리하는 것은 현실적으로 불가능 하다.
*   하지만, 비즈니스 상에서 모델의 활용 중에 성능이 저하된다면 의사결정 및 경영상의 문제를 초래 할 수 있다.
*   하여, concept drift의 통계량을 측정하여 모델의 성능 상태를 유추하는 연구들이 이루어져 왔다.
*   하지만, concept drift의 측정값 (통계량)을 통해 precision, recall, accuracy 등 직접적으로 성능을 예측하는 연구는 많지 않다.
*   이번 연구에서는 concept drift의 측정값 (통계량)을 통해 직접적으로 precision, recall, accuracy 등 성능을 예측하는 성능 예측 모델(PPM, Performance prediction model)을 제안한다.












# Proposed Method

1.   Feature의 importanc를 측정하는 Lime 알고리즘은 reference data를 기반으로 가상의 dataset을 임의 생성(permutation)하여 model.predict에 의한 결과 (Y)를 대조 분석하여 importance를 추정(실험의 시행에 따른 statistics에 해당) 한다.
2.   즉, dataset의 feature중 x1의 값이 변하여도 Y의 변동이 없다면 x1의 importance는 낮은 것이다. 
3.   실험 dataset에서 X1의 변동량 대비 Y의 변동량을 통계적으로 측정하여 importance를 추정하는 것이다.
4.  유사한 원리로 data drift  model performance (accuracy) 모델을 생성할 수 있을 것이다.
5.   즉, 전체 Trainset에서 N-fold class-validation의 개념으로 reference dataset과 inference dataset을 임의로 나누고, 
6.   reference 의 feature x1 대비 inference의 feature x1의 drift statistics 을 측정하고 (이때, x2,x3… 모든 feature의 drift statistics 이 측정될 것임) 이때의 inference dataset의 accuracy를 측정하여 y값으로 설정하고
7.   각 feature의 drift statistics 를 x로 설정하여 regressio에 의해 modeling (학습) 한다.
8.   물론, trainset 내의 분포, 범위라는 한계성을 갖지만,  직접적인 model 성능 추정이 가능할 것이다.









## PPM1 : feature의 drift statistics 로부터 직접 model의 성능을 추정한다.

In [ ]:
trainset : y=f(x)
base_score = Score of base model
scores = []
feature_drift = []
split_ratio = 0.1
for n fold:
    y_ref, y_inf, x_ref, x_inf = split(y, x, split_ratio)
    inf_score = score(y_inf, predict(x_inf))
    x_drift = drift(x, x_inf)
    scores.add(inf_score)
    feature_drift.add(x_drift)
        
perfomance_predict_model_1 = train(scores, feature_drift)

x_live = Inference data from time1 to time2
x_live_drift = drift(x, x_live)
predict_score = perfomance_predict_model_1(x_live_drift)

## PPM2 : feature의 drift statistics 로부터 base성능 대비 추정 성능의 차이를 추정한다.

In [ ]:
trainset : y=f(x)
base_score = Score of the model
base_feature_drift = The drift statistics of the feature that measured the base score 
scores = []
feature_drift = []
split_ratio = 0.1
for n fold:
    y_ref, y_inf, x_ref, x_inf = split(y, x, split_ratio)
    inf_score = score(y_inf, predict(x_inf))
    x_drift = drift(x, x_inf)
    scores.add(base_score - inf_score)
    feature_drift.add(base_feature_drift - x_drift)
        
perfomance_predict_model_2 = train(scores, feature_drift)

x_live = Inference data from time1 to time2
x_live_drift = drift(x, x_live)
predict_score = base_score - socre_predict_model_from_drift(base_feature_drift - x_live_drift)



# 실험 계획

*   Data
> 미국의 개인 대출 기업 ‘Lending Club’의 공개 데이터 사용<BR>
> 2007~2017년까지 총 2,260,701건의 데이터, 대출 건당 실제 ‘default’ 여부 포함
151개 Column<BR>
> 데이터 정제 후, 81개 Features, 120만건 사용<BR>

* 예측 알고리즘
> Random-forest 알고리즘<BR>
> Deep-learning 알고리즘

* 실험
> 2010년 data를 기준으로 모델 학습<BR>
>2011년~2017년의 데이터를 2010년 학습데이터의 건수 (8000건) 단위로 drift 측정
모델의 예측 성능과 실 성능을 비교<BR>

* 실험의 변수
> PPM1, PPM2<BR>
> Split_ratio = 0.1, 0.3, 0.5<BR>
> 예측 알고리즘 : random_forest, deep_learning<BR>

* 측정 지표
> 예측 알고리즘의 성능 지표 : precision<BR>
> PPM의 성능 지표<BR>
    >>Mean Absolute Error (MAE)<BR>
    >>Mean Squared Error (MSE)<BR>
    >>Root Mean Squared Error (RMSE)<BR>
    >>R-Squared<BR>
    >>Adjusted R-squared<BR>


## 데이터 준비

In [ ]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')